# Análisis Exploratorio de los datos de Trocafone



### Propuestas


## Configuracion inicial

In [ ]:
#Importo librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline
plt.style.use('default')
sns.set()

In [ ]:
#Funciones auxiliares
def mostrar_porcentaje_barplot(ax):
    suma = 0
    for p in ax.patches:
        suma += p.get_height()
    for p in ax.patches:
        ax.annotate(str(np.round(100 *(p.get_height() / suma),decimals=2)) + "%", (p.get_x()+p.get_width()/2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')
def mostrar_valores_barplot(ax):
    for p in ax.patches:
        ax.annotate(np.round(p.get_height(),decimals=2), (p.get_x()+p.get_width()/2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

In [ ]:
#Cargo el DataFrame
df = pd.read_csv('events.csv', low_memory = False, parse_dates = ['timestamp'], infer_datetime_format = True,
                    dtype = {'event': 'category','condition': 'category','storage': 'category', 'color': 'category', 'staticpage': 'category', 'campaign_source': 'category', 'search_engine': 'category', 'channel': 'category', 'new_vs_returning': 'category', 'region': 'category', 'country': 'category', 'device_type': 'category'})
df.head()

## Chequeos de integridad y calidad de los datos


In [ ]:
df.info()

Como podemos ver, todas las filas especifican 'timestamp', 'event' y 'person'

### Analizamos los valores que pueden tomar las columnas categoricas

In [ ]:
valores = df['condition'].unique()
for valor in valores:
    print(valor, end = ', ')

In [ ]:
valores = df['storage'].unique()
for valor in valores:
    print(valor, end = ', ')

In [ ]:
valores = df['channel'].unique()
for valor in valores:
    print(valor, end = ', ')

In [ ]:
valores = df['staticpage'].unique()
for valor in valores:
    print(valor, end = ', ')

## Analisis tipos de evento

In [ ]:
colUsadasEventos = df.groupby(by = 'event').count()
colUsadasEventos

In [ ]:
columns = list(df)
for event, row in colUsadasEventos.iterrows():
    print(event, end = ':')
    
    for column in columns:
        if(row.get(column) != 0):
            print(' ' + column, end = ',')
            
    print()

## Columnas usadas por cada evento

Todos los eventos contienen informacion sobre **'timestamp'**, **'event'** y **'person'**, y ademas utilizan las siguientes columnas:

**ad campaign hit:**
    url, campaign_source

**brand listing:**
    skus

**checkout:** 
    sku, model, condition, storage, color

**conversion:**
    sku, model, condition, storage, color

**generic listing:**
    skus

**lead:**
    model

**search engine hit:** 
    search_engine

**searched products:** 
    skus, search_term

**staticpage:** 
    staticpage

**viewed product:** 
    sku, model, condition, storage, color

**visited site:** 
    channel, new_vs_returning, city, region, country, device_type, screen_resolution, operating_system_version, browser_version


## Analisis geográfico

In [ ]:
#Solo los eventos "visited site" especifican pais
visitas = df.loc[df["event"] == "visited site", ['person', 'country', 'city', 'region']]

In [ ]:
visitas['country'].value_counts().head(10)

La mayor parte de las visitas provienen de Brasil, por lo que nos limitaremos a analizar a la audiencia brasileña

In [ ]:
visitas_brasil = visitas[visitas['country'] == 'Brazil']
#filtramos una visita por usuarios unicos, si un usuario entro muchas veces desde una ciudad la contaremos como una
# y si entro desde muchas ciudades, contaremos una visita por ciudad
visitas_brasil = visitas_brasil.drop_duplicates()

visitas_brasil['region'].value_counts().head()

In [ ]:
#Comprobamos que no especifique ciudad para region de tipo "Unknown"
visitas_brasil[visitas_brasil['region'] == 'Unknown']['city'].value_counts().head()

In [ ]:
#Filtramos los 'Unknown'
visitas_brasil = visitas_brasil[visitas_brasil['region'] != 'Unknown']
ax = visitas_brasil['region'].value_counts().head(15).plot('bar')
ax.set(xlabel='Region', ylabel='Cantidad de usuarios')
ax.set_title('Usuarios por region')
mostrar_porcentaje_barplot(ax)
sns.set(rc={'figure.figsize':(14.7,8.27)})

In [ ]:
visitas_brasil = visitas_brasil[visitas_brasil['city'] != 'Unknown']
ax = visitas_brasil['city'].value_counts().head(15).plot('bar')
ax.set(xlabel='Ciudad', ylabel='Cantidad de usuarios')
ax.set_title('Usuarios por ciudad')
mostrar_porcentaje_barplot(ax)
sns.set(rc={'figure.figsize':(14.7,8.27)})

### Falta meterle un Basemap!!!!!!!!!!!!!!!!!!!

## Captacion de usuarios

In [ ]:
personas_new_ret = df.loc[df["event"] == "visited site", ['person', 'new_vs_returning']]
personas_new_ret['returning_hits'] = personas_new_ret['new_vs_returning'] == 'Returning'
personas_new_ret.drop(columns='new_vs_returning', inplace=True)
personas_new_ret.head()

In [ ]:
personas_ret_hits = personas_new_ret.groupby(by = 'person').agg({'returning_hits': np.count_nonzero})
personas_ret_hits.head(10)

In [ ]:
x = personas_ret_hits['returning_hits']
x.value_counts().head(10)

In [ ]:
sns.distplot(x, hist_kws={'log':True}, kde = False, axlabel = 'Cantidad de returning_hits').set_title('Cantidad de usuarios según returning_hits')

La mayoría de los usuarios ingresa al sitio una única vez

In [ ]:
cant_returns = x.value_counts()
cero_returns = cant_returns[0]
uno_diez_returns = cant_returns[1:11].values.sum()
diez_mas_returns = cant_returns[11:].values.sum()
datos_returns = [cero_returns, uno_diez_returns, diez_mas_returns]
x_labels = ['1', '2 - 10', '10+']
ax = sns.barplot(x = x_labels, y = datos_returns)
ax.set(xlabel='Cantidad de visitas al sitio', ylabel='Cantidad de Usuarios', title = 'Cantidad de usuarios segun veces que visitaron el sitio')

La mayoria de los usuarios visita el sitio una única vez, mientras que solo una pequeña proporción volvió al sitio mas de 10 veces. Es posible que un mismo usuario entre desde dispositivos diferentes.

## Analisis temporal

In [ ]:
#Analizamos periodo de los datos
df['timestamp'].dt.year.value_counts()

Todos los datos son de 2018

In [ ]:
df['timestamp'].dt.month_name().value_counts()

De los primeros dos trimestres del año

In [ ]:
#Vamos a hacer un analisis sobre los dias en los que se produjeron los eventos
dias_semana = df['timestamp'].dt.weekday_name
dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dias_semana = pd.Categorical(dias_semana, categories = dias_ordenados, ordered = True)
g = dias_semana.value_counts().plot('bar')
g.set_title('Cantidad de eventos por dia de la semana')

Vemos que durante el fin de semana el trafico es menor

In [ ]:
tiempo_eventos = pd.DataFrame()
tiempo_eventos['event'] = df['event']
tiempo_eventos['dia_semana'] = df['timestamp'].dt.weekday_name
tiempo_eventos['dia'] = df['timestamp'].dt.day
tiempo_eventos['hora'] = df['timestamp'].dt.hour
tiempo_eventos['mes'] = df['timestamp'].dt.month_name()

hora_weekday_visitas = tiempo_eventos[tiempo_eventos['event'] == 'visited site'].groupby('dia_semana')['hora'].value_counts().to_frame()
hora_weekday_visitas = hora_weekday_visitas.rename(columns = {'hora':'cant_casos'})
hora_weekday_visitas = hora_weekday_visitas.reset_index('hora')
hora_weekday_visitas.index = pd.CategoricalIndex(hora_weekday_visitas.index, categories= dias_ordenados)
hora_weekday_visitas.sort_index(level=0, inplace=True)
hora_weekday_visitas = hora_weekday_visitas.reset_index()

dia_mes_visitas = tiempo_eventos[tiempo_eventos['event'] == 'visited site'].groupby('mes')['dia'].value_counts().to_frame()
dia_mes_visitas = dia_mes_visitas.rename(columns = {'dia':'cant_casos'})
dia_mes_visitas = dia_mes_visitas.reset_index('dia')
dia_mes_visitas.index = pd.CategoricalIndex(dia_mes_visitas.index, categories= ['January', 'February', 'March', 'April', 'May', 'June'])
dia_mes_visitas.sort_index(level=0, inplace=True)
dia_mes_visitas = dia_mes_visitas.reset_index()

In [ ]:
pivot = hora_weekday_visitas.pivot_table(index = 'dia_semana', columns = 'hora', values = 'cant_casos')
cmap = sns.cm.rocket_r
ax = sns.heatmap(data = pivot, cmap = cmap)
ax.set(xlabel='Hora', ylabel='Dia de la semana', title = 'Visitas segun hora del dia')

Durante el fin de semana la cantidad de visitas es menor, mientras que la mayoria de las visitas se realizan entre las 12 y las 2

In [ ]:
pivot = dia_mes_visitas.pivot_table(index = 'dia', columns = 'mes', values = 'cant_casos')
cmap = sns.cm.rocket_r
sns.set(rc={'figure.figsize':(10.7,9.27)})
ax = sns.heatmap(data = pivot, cmap = cmap, annot = True, fmt=".0f")
ax.set(xlabel='Mes', ylabel='Dia', title = 'Visitas 2018')

Las visitas crecieron aceleradamente a lo largo del año, logrando un despegue a partir de mayo. Analizaremos las causas de esto mas adelante

## Origen de las visitas

In [ ]:
visitas = df.loc[df["event"] == "visited site", ['timestamp', 'person', 'channel', 'new_vs_returning', 'city', 'region', 'country', 'device_type', 'screen_resolution', 'operating_system_version', 'browser_version']]
colors = {'Paid': 'navy', 'Direct': 'g', 'Organic': 'coral', 'Referral': 'm', 'Social': 'goldenrod', 'Email': 'k', 'Unknown': 'grey'}

In [ ]:
valores = visitas['channel'].value_counts()
ax = valores.plot(kind = 'bar', color=[colors.get(i) for i in valores.index])
ax.set_title('Origen de las visitas')
ax.set_ylabel('Cantidad')
ax.set_xlabel('Channel')
plt.xticks(rotation=0)
mostrar_porcentaje_barplot(ax)

### Origen de usuarios nuevos

In [ ]:
valores = visitas[visitas['new_vs_returning'] == 'New']['channel'].value_counts()
ax = valores.plot(kind = 'bar', color=[colors.get(i) for i in valores.index])
ax.set_title('Origen de las visitas de usuarios nuevos')
ax.set_ylabel('Cantidad')
ax.set_xlabel('Channel')
plt.xticks(rotation=0)
mostrar_porcentaje_barplot(ax)

### Origen usuarios no nuevos

In [ ]:
valores = visitas[visitas['new_vs_returning'] == 'Returning']['channel'].value_counts()
ax = valores.plot(kind = 'bar', color=[colors.get(i) for i in valores.index])
ax.set_title('Origen de las visitas de usuarios no nuevos')
ax.set_ylabel('Cantidad')
ax.set_xlabel('Channel')
plt.xticks(rotation=0)
mostrar_porcentaje_barplot(ax)

<font size=3>Vemos que el canal pago es el que mas visitas atrae, no solo para usuarios nuevos sino tambien para usuarios que ya habian entrado al sitio. Por otro lado, se puede ver como el canal "Direct" (tipeado de URL, acceso mediante favoritos y similares) toma protagonismo para visitas de usuarios que ya conocen el sitio. Pero esto puede ser causado por un pequeño numero de usuarios que visitan repetidas veces el sitio</font>

<font size=3>Por lo tanto ahora tomaremos solo un metodo predominante de acceso al sitio por usuario, para usarios no nuevos</font>

In [ ]:
valores = visitas[visitas['new_vs_returning'] == 'Returning'].groupby('person')['channel'].agg(lambda x: x.value_counts().index[0]).value_counts()

ax = valores.plot(kind = 'bar', color=[colors.get(i) for i in valores.index])
ax.set_title('Canal predominante de ingreso al sitio para usuarios no nuevos')
ax.set_ylabel('Cantidad de usuarios')
ax.set_xlabel('Evento predominante')
plt.xticks(rotation=0)
mostrar_porcentaje_barplot(ax)

Vemos que en este caso el canal predominante de ingreso al sitio para usuarios no nuevos sigue siendo el canal pago, aunque con menor proporcion que para usuarios nuevos. El ingreso mediante "referrals" de otros sitios queda en segundo lugar, teniendo una proporcion mucho mayor que para usuarios nuevos.

## Analisis publicitario

In [ ]:
df_indice_en_persona_evento = df.set_index(['person','event'])

In [ ]:
dif_primer_ultimo_evento = df_indice_en_persona_evento.groupby('person').agg({'timestamp':['max', 'min']})
dif_primer_ultimo_evento['diferencia'] = dif_primer_ultimo_evento['timestamp']['max'] - dif_primer_ultimo_evento['timestamp']['min']
dif_primer_ultimo_evento.drop(columns='timestamp', inplace= True)
#mejora notoria de tiempo de ejecucion

In [ ]:
dif_primer_ultimo_evento = dif_primer_ultimo_evento.loc[dif_primer_ultimo_evento['diferencia']>'1 day'].sort_values(by = 'diferencia')
pd.concat([dif_primer_ultimo_evento.head(), dif_primer_ultimo_evento.tail()])

# Quiero ver Que campaña publicitaria lleva a mas ventas

In [ ]:
personas_por_publicidad = {}
    
ad_campaigns = df.loc[(df['event']=='ad campaign hit') | (df['event']=='conversion')].drop_duplicates()#[['person','campaign_source']].drop_duplicates().set_index('campaign_source')
ad_campaigns

In [ ]:
ad_campaigns_cool = df.loc[(df['event']=='ad campaign hit'),['person','campaign_source']].drop_duplicates()
ad_campaigns_cool.head()

In [ ]:
personas_que_compraron = df.loc[df['event']=='conversion',['person']].drop_duplicates()
personas_que_compraron['compro'] = True
personas_que_compraron.head()

In [ ]:
compras_por_campania = pd.merge(ad_campaigns_cool, personas_que_compraron, on='person', how='left')
compras_por_campania.head(10)

In [ ]:
compras2 = compras_por_campania.groupby('campaign_source').agg({'person': 'count','compro':'count'})
compras2.head()

In [ ]:
g = compras2['person'].sort_values(ascending=False).head().plot('bar')
g.set_title('Hits por campaña')
g.set_xlabel('Campaña publicitaria')
g.set_ylabel('Cantidad de hits')

In [ ]:
g2 = compras2['compro'].sort_values(ascending=False).plot('bar')
g2.set_title('Cantidad de compras realizadas por campaña')

In [ ]:
#QUITAMOS LOS QUE TIENEN MENOS DE 10 HITS PARA NO DEFORMAR
compras2 = compras2[compras2["person"] > 10]
compras2['porcentaje'] = 100* (compras2['compro']/compras2['person'])
g3 = compras2['porcentaje'].sort_values(ascending=False).plot('bar')
g3.set_title('Porcentaje de compras logradas por campaña publicitaria')

Hay una "campaña publicitaria" llamada onsite que debería ser eliminada del data frame

## Analisis por dispositivo 

In [ ]:
df['device_type'].value_counts()

In [ ]:
dispositivos_registrados = df.loc[(df['event']=='visited site'),['person','device_type','new_vs_returning']]
dispositivos_nuevos = dispositivos_registrados.loc[dispositivos_registrados['new_vs_returning']=='New']

g_dispositivos_nuevos = dispositivos_nuevos['device_type'].value_counts().plot('bar')
g_dispositivos_nuevos.set_title('Dispositivos de nuevas visitas [New Hits]')
g_dispositivos_nuevos.set_xlabel('Tipo de dispositivo')
g_dispositivos_nuevos.set_ylabel('Cantidad de personas')

Los unicos dispositivos relevantes son los de Smartphones y Computadoras

In [ ]:
#Filtro las personas entraron por primera vez por su celular
personas_nuevas_smartphone = dispositivos_nuevos.loc[(dispositivos_nuevos['device_type']=='Smartphone'),['person']]

#Filtro paralelamente las personas que volvieron a entrar desde sus PCs
personas_returning_pc = dispositivos_registrados.loc[(dispositivos_registrados['device_type']=='Computer') & (df['new_vs_returning']=='Returning'),['person']]
personas_returning_pc['Volvieron_en_PC'] = True

In [ ]:
personas_new_Smartphone_ret_PC = pd.merge(personas_nuevas_smartphone, personas_returning_pc, on='person', how='left').drop_duplicates()
personas_new_Smartphone_ret_PC['Volvieron_en_PC'].fillna(False,inplace=True)
personas_new_Smartphone_ret_PC.describe()

#### Veo la cantidad de personas que entraron por primera vez a la página desde sus celulares y volvieron desde sus PCs

In [ ]:
cantidad_personas_que_volvieron_enPC = personas_new_Smartphone_ret_PC['Volvieron_en_PC'].value_counts()
cantidad_personas_que_volvieron_enPC


In [ ]:
g_personas_ret_PC = cantidad_personas_que_volvieron_enPC.plot('bar')
g_personas_ret_PC.set_title('Personas que vuelven a la pagina desde la PC luego de entrar por primera vez desde Smartphone')
g_personas_ret_PC.set_xlabel('Vuelve a la pagina')
g_personas_ret_PC.set_ylabel('Cantidad de personas')

Se nota que no es representativa el numero de personas que entran primero a la pagina por sus celulares y luego vuelven a entrar desde la PC

#### Analizamos si de las personas que vuelven a entrar desde la PC (y que primero ingresaron con sus Smartphones) terminan realizando alguna compra

In [ ]:
personas_vuelven_y_compran = pd.merge(personas_new_Smartphone_ret_PC, personas_que_compraron, on='person', how='left').drop_duplicates()
personas_vuelven_y_compran['compro'].fillna(False,inplace=True)
personas_vuelven_y_compran.loc[personas_vuelven_y_compran['Volvieron_en_PC']==True]['compro'].value_counts(normalize = True)

In [ ]:
g5 = personas_vuelven_y_compran.loc[personas_vuelven_y_compran['Volvieron_en_PC']==True]['compro'].value_counts().plot('bar',)
g5.set_title("Compras de personas que entraron por primera vez en Smarphone, y luego volvieron en PC ")
g5.set_xlabel("Compraron")
g5.set_ylabel("Cantidad de personas")

Podemos ver que de las personas que volvieron a la pagina desde sus PCs luego de entrar desde sus SmartPhones, un 28,4% realizo compras

## IDEAS
Se puede analizar desde el primer evento (el visited_site con New) de cada usuario, tomando este evento como el cero; cuanto tiempo le lleva o tarda en realizar mas acciones en la pagina

## Los usuarios que hicieron checkout. ¿Concretaron la conversión?

In [ ]:
checkouts = df.loc[df['event'] == 'checkout',["sku","timestamp","person","model","condition","storage","color"]]
conversiones = df.loc[df['event'] == 'conversion',["sku","timestamp","person","model","condition","storage","color"]]

### Observemos la evolución de los checkouts en el tiempo

In [ ]:
checkouts_by_day = df.loc[df['event'] == 'checkout',["timestamp"]]
checkouts_by_day["Fecha"] = checkouts_by_day["timestamp"].dt.date
checkouts_by_day["Semana"] = checkouts_by_day["timestamp"].dt.week
checkouts_by_day.drop("timestamp", axis=1, inplace= True)
checkouts_by_day["Checkouts"] = 1

checkouts_by_week = checkouts_by_day.groupby(by="Semana").agg({"Fecha": "first", "Checkouts": "count"}).set_index('Fecha')
checkouts_by_week = checkouts_by_week.iloc[0:-1] # elimino la ultima semana porque quedo con menos de 7 dias
checkouts_by_day = checkouts_by_day.groupby(by="Fecha")['Checkouts'].agg("count")

ax = checkouts_by_week.plot()
ax.set(xlabel='Semana', ylabel='Cantidad de checkouts')
ax.set_title('Checkouts semanales')

Aparentemente, algo ocurrió a mitad de Mayo. Haciendo zoom podemos ver que el 14 comenzaron a aumentar los checkouts.

In [ ]:
acotado = checkouts_by_day.loc[pd.to_datetime("2018-05-11").date() : pd.to_datetime("2018-05-18").date()]

ax = acotado.plot(marker=".",markersize="20")
ax.set(xlabel='Dia', ylabel='Cantidad de checkouts')
ax.set_title('Checkouts 11/05 al 17/05')

Ahora analicemos la evolución de las conversiones.

In [ ]:
conversiones_by_day = df.loc[df['event'] == 'conversion',["timestamp"]]
conversiones_by_day["Fecha"] = conversiones_by_day["timestamp"].dt.date
conversiones_by_day["Semana"] = conversiones_by_day["timestamp"].dt.week
conversiones_by_day.drop("timestamp", axis=1, inplace= True)
conversiones_by_day["Conversiones"] = 1
#conversiones_by_day = conversiones_by_day.groupby(by="Fecha").agg("count")
conversiones_by_day = conversiones_by_day.groupby(by="Semana").agg({"Fecha": "first", "Conversiones": "count"}).set_index('Fecha')
conversiones_by_day = conversiones_by_day.iloc[0:-1]
conversiones_by_day.plot()

Veamos si hay correlación entre el aumento de checkouts y las conversiones.

In [ ]:
#checkouts_y_conversiones = conversiones_by_day.join(checkouts_by_day,on="Fecha")
#checkouts_y_conversiones.plot()
fig, ax1 = plt.subplots()

color = 'tab:green'
ax1.set_xlabel('Semana')
ax1.set_ylabel('Cantidad de checkouts', color=color)
ax1.plot(checkouts_by_week, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Cantidad de conversiones', color=color)
ax2.plot(conversiones_by_day, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()

Se observa un aumento significativo tanto de conversiones como de checkouts en la segunda semana de mayo, pero de ahi en adelante solo los checkouts siguieron aumentando mientras que las conversiones disminuyeron.

### Marcas con mas checkouts y conversiones

In [ ]:
def obtener_marca(serie_de_modelo):
    # La marca es siempre la primer palabra del modelo
    r = []
    for modelo in serie_de_modelo.tolist():
        if pd.isna(modelo):
            r.append("Unknown")
        else:
            r.append(modelo.split()[0])
    return r

In [ ]:
checkouts["Marca"] = obtener_marca(checkouts["model"])
checkouts = checkouts.loc[checkouts["Marca"] != "Unknown"] # Remuevo datos erroneos
checkouts["Checkouts"] = 1
checkouts_by_marca = checkouts[["Marca","Checkouts"]].groupby(by=["Marca"]).agg("count")\
    .sort_values(by="Checkouts",ascending=False)
checkouts_by_marca.plot(kind="bar")

In [ ]:
conversiones["Marca"] = obtener_marca(conversiones["model"])
conversiones = conversiones.loc[conversiones["Marca"] != "Unknown"] # Remuevo datos erroneos
conversiones["Conversiones"] = 1
conversiones_by_marca = conversiones[["Marca","Conversiones"]].groupby(by=["Marca"]).agg("count")\
    .sort_values(by="Conversiones",ascending=False)
conversiones_by_marca.plot(kind="bar")

In [ ]:
marca_stats = conversiones_by_marca.join(checkouts_by_marca).sort_values(by="Checkouts",ascending=False)
marca_stats["CheckoutsSinConversion"] = marca_stats["Checkouts"] - marca_stats["Conversiones"]
marca_stats.drop("Checkouts", axis=1, inplace= True)
marca_stats.plot(kind="bar",stacked=True)

In [ ]:
#analizar
pd.crosstab(checkouts['condition'],checkouts['Marca'])